In [1]:
import torch
from torch import nn


In [2]:
class CRF(nn.Module):
    """
    这个主要是线性链条件随机场
    nb_labels(int): 标注序列的总的类别数；
    bos_tag_id(int): 表示句子起始符号对应的id
    eos_tag_id(int): 表示句子结束符号对应的id
    batch_first(bool): 是个布尔变量，表示向量的第一维是否表示batch 维度
    """
    
    
    def __init__(self, nb_labels, bos_tag_id, eos_tag_id, batch_first=True):
        super().__init__()
        self.nb_labels = nb_labels
        self.BOS_TAG_ID = bos_tag_id
        self.EOS_TAG_ID = eos_tag_id
        self.batch_first = batch_first
        self.transitions = nn.Parameter(torch.empty(self.nb_labels, self.nb_labels))
        self.init_weights()
        
    def init_weights(self):
        nn.init.uniform_(self.transitions, -0.1,0.1)
        # 将transitions矩阵，也就是lable与label之间的转移矩阵进行随机初始化，(-0.1,0.1)之间的均匀分布
        self.transitions.data[:, self.BOS_TAG_ID] = -10000.0
        # 因为不允许有存在label转移到下一个label为bos，所以将他们的值设置为-10000，因为exp(-10000)接近于0
        self.transitions.data[self.EOS_TAG_ID,:] = -10000.0
        # 同理，不允许eos转移到任意的下一个label，所以也将他们的值设置为-10000
        
    
    def forward(self, emissions, tags, mask=None):
        """
        计算负对数似然损失
        """
    nll = -self.log_likelihood(emissions, tags, mask=mask)
    return nll

    def log_likelihood(self, emissions, tags, mask=None):
        """
        计算损失函数
        emissions(torch.tensor): 维度是(batch_size, seq_len, nb_labels)，是模型输出的句子的每个位置对于每个label的score，
                    如果bach_first为false的话，维度就是(seq_len, batch_size, nb_labels)。下同
        
        tags(torch.tensor): 维度是(batch_size,seq_len), 句子序列的gold labels
        
        mask(torch.tensor): 维度是(batch_size, seq_len)，表示序列中的有效位置
        
        returns: torch.tensor,维度是(batch_size), 对于每一个句子的对数似然值
        """
        
        if not self.batch_first:
            emissions = emissions.transpose(0,1)
            tags = tags.transpose(0,1)
            
        if mask is None:
            mask = torch.ones(emissions.shape[:2], dtype=torch.float)
            # emissions.shape[:2]，返回的结果是(batch_size, seq_len)，所以mask的维度就是(batch_size, seq_len)
            
        scores = self._compute_scores(emissions, tags, mask=mask)
        partition = self._compute_log_partition(emissions, mask=mask)
        return torch.sum(scores - partition)
    
    
    def _compute_scores(self, emissions, tags, mask):
        """
        对于每个batch，用emissions和tags计算分值；
        Args:
            emissions(torch.tensor): (batch_size, seq_len, nb_labels)
            tags(torch.tensor): (batch_size, seq_len)
            mask(torch.tensor): (batch_size, seq_len)
            
        returns:
            torch.tensor: 对于每个batch内的每个seq的分值， 维度是(batch_size)
        """
        batch_size, seq_length = tags.shape
        scores = torch.zeros(batch_size)
        
        # 保存每个seq的第一个tag和最后一个tag，留着进行从(bos -> first_tag)和(last_tag -> eos)的转换
        first_tags = tags[:,0]
        last_valid_idx = mask.int().sum(1)-1
        last_tags = tags.gather(1, last_valid_idx.unsqueeze(1)).squeeze()
        
        #计算从bos到first_tags之间转换的分值
        t_scores = self.transitions[self.BOS_TAG_ID, first_tags]
        
        #计算emission对于first_tags的分值，也就是去emission矩阵中取出下标为first_tags的值
        e_scores = emissions[:,0].gather(1, first_tags.unsqueeze(1)).squeeze()
        
        # 对于一个词的分值是把t_scores和e_scores加起来
        scores += e_scores + t_scores
        
        # 现在就是对于剩下的每一个词都做这个运算
        for i in range(1, seq_length):
            is_valid = mask[:i]
            
            previous_tags = tags[:, i-1]
            current_tags = tags[:, i]
            
            # 计算t_score和e_score
            e_scores = emissions[:,i].gather(1, current_tags.unsqueeze(1)).squeeze()
            t_scores = self.transitions[previous_tags, current_tags]
            
            # 应用mask
            e_scores = e_scores * is_valid
            t_scores = t_scores * is_valid
            
            scores += e_scores + t_scores
            
        scores += self.transitions[last_tags, self.EOS_TAG_ID]
        
        return scores
        
    